In [77]:
# Librerias a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from textblob import TextBlob
#from wordcloud import WordCloud
import re
from matplotlib import style
plt.style.use('ggplot')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('spanish'))
import nltk 
import spacy
from spacy import displacy
import csv

In [35]:
# Importamos los datos
tweets = pd.read_csv("Plan_integral_movilidad.csv")
tweets.columns

Index(['text', 'favorited', 'favoriteCount', 'replyToSN', 'created',
       'truncated', 'replyToSID', 'id', 'replyToUID', 'statusSource',
       'screenName', 'retweetCount', 'isRetweet', 'retweeted', 'longitude',
       'latitude'],
      dtype='object')

Observamos variables como el texto del tweet, su fecha de creación, si fue rtweet, el conteo de los rweets y el usuario del tweet

A través del método de web scraping en la API de Twitter acerca de palabras relacionadas al proyecto de movilidad en el sur de la ciudad se encontraron 1735 tweets a los cuales se procedera aplicar procesamiento de lenguaje natural y análisis de sentimiento para conocer la percepcíón de la ciudadanía sobre el proyecto.

In [37]:
#Dataframe con los tweets como texto
text_tweets = pd.DataFrame(tweets['text'])
text_tweets.head()

,text
0,¿Ya vieron todo lo que incluye el Plan Integra...
1,RT @JorgeAldanaB: En nuestro compromiso por me...
2,"RT @quierotv_gdl: ""Solución López Mateos"" \n??..."
3,"""Solución López Mateos"" \n?? Conoce la estrate..."
4,RT @PpopularElche: Proponemos un Plan de Movil...


In [39]:
#Realizamos limpieza de información sobre el texto de los tweets
def limpiar_texto(text):
    text = text.lower()
    text = re.sub(r'@[A-Za-z09]+','',text)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'rt[\s]+','',text)
    text = re.sub(r'https:\/\/?','',text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [40]:
text_tweets['text'] = text_tweets['text'].apply(limpiar_texto)
text_tweets.head()

,text
0,vieron incluye plan integral movilidad present...
1,compromiso mejorar movilidad capital realizado...
2,_gdl solución lópez mateos conoce estrategia m...
3,solución lópez mateos conoce estrategia movili...
4,proponemos plan movilidad integral municipal c...


In [41]:
text_tweets = text_tweets.drop_duplicates('text')
text_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 1734
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    716 non-null    object
dtypes: object(1)
memory usage: 11.2+ KB


In [42]:
stemmer= PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [43]:
text_tweets['text'] = text_tweets['text'].apply(lambda x: stemming(x))
text_tweets.head()

,text
0,vieron incluye plan integral movilidad present...
1,compromiso mejorar movilidad capital realizado...
2,_gdl solución lópez mateos conoce estrategia m...
3,solución lópez mateos conoce estrategia movili...
4,proponemos plan movilidad integral municipal c...


NTLK

In [46]:
#tokenizar el texto en palabras y en oraciones:
tokens = texto.apply(nltk.word_tokenize)
tokens

0       [vieron, incluye, plan, integral, movilidad, p...
1       [compromiso, mejorar, movilidad, capital, real...
2       [_gdl, solución, lópez, mateos, conoce, estrat...
3       [solución, lópez, mateos, conoce, estrategia, ...
4       [proponemos, plan, movilidad, integral, munici...
                              ...                        
1729    [gobierno, macuspana, secretaría, ordenamiento...
1730    [cali, hará, parte, creación, primera, área, m...
1732    [cali, hará, parte, creación, primera, área, m...
1733    [proyectos, renovación, infraestructura, públi...
1734    [proyectos, renovación, infraestructura, públi...
Name: text, Length: 716, dtype: object

In [47]:
#Eliminamos las stopwords (palabras sin significado como "y", "o", etc.):
from nltk.corpus import stopwords

stopwords_sp = stopwords.words('spanish')

texto['tokens_sin_stopwords'] = tokens.apply(lambda x: [palabra for palabra in x if palabra.lower() not in stopwords_sp])

In [48]:
texto['tokens_sin_stopwords'] 

0       [vieron, incluye, plan, integral, movilidad, p...
1       [compromiso, mejorar, movilidad, capital, real...
2       [_gdl, solución, lópez, mateos, conoce, estrat...
3       [solución, lópez, mateos, conoce, estrategia, ...
4       [proponemos, plan, movilidad, integral, munici...
                              ...                        
1729    [gobierno, macuspana, secretaría, ordenamiento...
1730    [cali, hará, parte, creación, primera, área, m...
1732    [cali, hará, parte, creación, primera, área, m...
1733    [proyectos, renovación, infraestructura, públi...
1734    [proyectos, renovación, infraestructura, públi...
Name: text, Length: 716, dtype: object

SpaCy

In [75]:
import es_core_news_sm
nlp = es_core_news_sm.load()

In [82]:
def spacy_analyzer(text):
    doc = nlp(text)
    return doc

In [90]:
doc = texto['textos_analizados'] = text_tweets["text"].apply(spacy_analyzer)

In [94]:
#Extracción de las distintas entidades nombradas en el texto (NER)
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for entity in doc.ents:
        entities.append((entity.text, entity.label_))
    return entities
texto['entidades_nombradas'] = text_tweets['text'].apply(extract_entities)
texto['entidades_nombradas'] 

0                                             []
1                                             []
2                                   [(gdl, ORG)]
3                 [(solución lópez mateos, PER)]
4                                             []
                          ...                   
1729                 [(gobierno macuspana, PER)]
1730     [(cali hará, PER), (colombiaamso, LOC)]
1732     [(cali hará, PER), (colombiaamso, LOC)]
1733                          [(proyectos, PER)]
1734    [(proyectos, PER), (tcoohus5zmkta, LOC)]
Name: text, Length: 716, dtype: object

In [101]:
def analyze_syntax(text):
    doc = nlp(text)
    return [(token.text, token.pos_, token.dep_, token.head.text) for token in doc]

text_tweets['analisis_sintactico'] = text_tweets['text'].apply(analyze_syntax)
text_tweets['analisis_sintactico'] 

0       [(vieron, VERB, ccomp, presentó), (incluye, VE...
1       [(compromiso, NOUN, ROOT, compromiso), (mejora...
2       [(_, PROPN, nsubj, conoce), (gdl, PROPN, flat,...
3       [(solución, PROPN, nsubj, conoce), (lópez, PRO...
4       [(proponemos, VERB, ROOT, proponemos), (plan, ...
                              ...                        
1729    [(gobierno, NOUN, nsubj, secretaría), (macuspa...
1730    [(cali, PROPN, nsubj, hará), (hará, VERB, ROOT...
1732    [(cali, PROPN, nsubj, hará), (hará, VERB, ROOT...
1733    [(proyectos, NOUN, nsubj, realiza), (renovació...
1734    [(proyectos, NOUN, nsubj, realiza), (renovació...
Name: analisis_sintactico, Length: 716, dtype: object

In [102]:
#Análisis de sentimiento del texto para evaluar la opinión de los ciudadanos al respecto del nuevo plan de movilidad
#0 netral, -1 negativa, +1 positivo
def analyze_sentiment(text):
    doc = nlp(text)
    return doc.sentiment

text_tweets['sentimiento'] = text_tweets['text'].apply(analyze_sentiment)
text_tweets['sentimiento']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1729    0.0
1730    0.0
1732    0.0
1733    0.0
1734    0.0
Name: sentimiento, Length: 716, dtype: float64

In [138]:
text_tweets['polaridad'] = text_tweets['text'].apply(lambda x: [token.sentiment for token in nlp(x)])
text_tweets['polaridad']

0           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
1           [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
1729    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1730    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1732    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1733    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1734    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: polaridad, Length: 716, dtype: object

In [118]:
def analyze_dependencies(text):
    doc = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in doc]

text_tweets['dependencias'] = text_tweets['text'].apply(analyze_dependencies)
text_tweets

,text,analisis_sintactico,sentimiento,entidades,dependencias
0,vieron incluye plan integral movilidad present...,"[(vieron, VERB, ccomp, presentó), (incluye, VE...",0.0,[],"[(vieron, ccomp, presentó), (incluye, ccomp, v..."
1,compromiso mejorar movilidad capital realizado...,"[(compromiso, NOUN, ROOT, compromiso), (mejora...",0.0,[],"[(compromiso, ROOT, compromiso), (mejorar, csu..."
2,_gdl solución lópez mateos conoce estrategia m...,"[(_, PROPN, nsubj, conoce), (gdl, PROPN, flat,...",0.0,"[(gdl, ORG)]","[(_, nsubj, conoce), (gdl, flat, _), (solución..."
3,solución lópez mateos conoce estrategia movili...,"[(solución, PROPN, nsubj, conoce), (lópez, PRO...",0.0,"[(solución lópez mateos, PER)]","[(solución, nsubj, conoce), (lópez, flat, solu..."
4,proponemos plan movilidad integral municipal c...,"[(proponemos, VERB, ROOT, proponemos), (plan, ...",0.0,[],"[(proponemos, ROOT, proponemos), (plan, obj, p..."
...,...,...,...,...,...
1729,gobierno macuspana secretaría ordenamiento ter...,"[(gobierno, NOUN, nsubj, secretaría), (macuspa...",0.0,"[(gobierno macuspana, PER)]","[(gobierno, nsubj, secretaría), (macuspana, am..."
1730,cali hará parte creación primera área metropol...,"[(cali, PROPN, nsubj, hará), (hará, VERB, ROOT...",0.0,"[(cali hará, PER), (colombiaamso, LOC)]","[(cali, nsubj, hará), (hará, ROOT, hará), (par..."
1732,cali hará parte creación primera área metropol...,"[(cali, PROPN, nsubj, hará), (hará, VERB, ROOT...",0.0,"[(cali hará, PER), (colombiaamso, LOC)]","[(cali, nsubj, hará), (hará, ROOT, hará), (par..."
1733,proyectos renovación infraestructura pública _...,"[(proyectos, NOUN, nsubj, realiza), (renovació...",0.0,"[(proyectos, PER)]","[(proyectos, nsubj, realiza), (renovación, amo..."


In [120]:
def lemmatize(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc]

text_tweets['lemmas'] = text_tweets['text'].apply(lemmatize)
text_tweets['lemmas']

0       [ver, incluir, plan, integral, movilidad, pres...
1       [compromiso, mejorar, movilidad, capital, real...
2       [_, gdl, solución, lópez, mateos, conocer, est...
3       [solución, lópez, mateos, conocer, estrategia,...
4       [proponer, plan, movilidad, integral, municipa...
                              ...                        
1729    [gobierno, macuspán, secretacer, ordenamiento,...
1730    [cali, hacer, parte, creación, primero, área, ...
1732    [cali, hacer, parte, creación, primero, área, ...
1733    [proyecto, renovación, infraestructura, públic...
1734    [proyecto, renovación, infraestructura, públic...
Name: lemmas, Length: 716, dtype: object

In [130]:
def recognize_idioms(text):
    doc = nlp(text)
    idioms = []
    for token in doc:
        if token.dep_ == 'compound' and token.head.pos_ == 'NOUN' and token.head.dep_ == 'dobj':
            idioms.append(token.text + ' ' + token.head.text)
    return idioms

text_tweets['idiomas'] = text_tweets['text'].apply(recognize_idioms)
text_tweets['idiomas']

0       []
1       []
2       []
3       []
4       []
        ..
1729    []
1730    []
1732    []
1733    []
1734    []
Name: idiomas, Length: 716, dtype: object

In [136]:
text_tweets['frases'] = text_tweets['text'].apply(lambda x: [sent.text for sent in nlp(x).sents])
text_tweets['frases'] 

0       [vieron incluye plan integral movilidad presen...
1       [compromiso mejorar movilidad capital realizad...
2       [_gdl solución lópez mateos conoce estrategia ...
3       [solución lópez mateos conoce estrategia movil...
4       [proponemos plan movilidad integral municipal ...
                              ...                        
1729    [gobierno macuspana secretaría ordenamiento te...
1730    [cali hará parte creación primera área metropo...
1732    [cali hará parte creación primera área metropo...
1733    [proyectos renovación infraestructura pública ...
1734    [proyectos renovación infraestructura pública ...
Name: frases, Length: 716, dtype: object